<a href="https://colab.research.google.com/github/SaideLeon/ClonarVOZ/blob/main/Graphql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
token="b567f2e6-d265-4659-bc58-04ed5b9aaad0"

In [ ]:
import requests
import json
import base64

# === Configurações ===
url = 'https://uploader.nativespeak.app/api/graphql'
project_name = 'MeuProjeto'
audio_file_path = 'audio.wav'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}

# Ler e codificar o arquivo de áudio
try:
    with open(audio_file_path, "rb") as f:
        audio_bytes = f.read()
        audio_base64 = base64.b64encode(audio_bytes).decode("utf-8")
except FileNotFoundError:
    print(f"Erro: arquivo '{audio_file_path}' não encontrado.")
    exit(1)

# Mutation para upload
upload_mutation = """
mutation($input: UploadFileInput!) {
  uploadFile(input: $input) {
    id
    name
    mimeType
    size
    url
  }
}
"""

upload_variables = {
    "input": {
        "project": project_name,
        "fileName": audio_file_path,
        "contentType": "audio/wav",
        "contentBase64": audio_base64
    }
}

try:
    upload_resp = requests.post(url, headers=headers, json={
        'query': upload_mutation,
        'variables': upload_variables
    })
    upload_resp.raise_for_status()
    upload_data = upload_resp.json()['data']['uploadFile']
    print("✅ Upload realizado com sucesso:")
    print(json.dumps(upload_data, indent=2))
except requests.exceptions.RequestException as e:
    print(f"Erro no upload: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")
    exit(1)

# Salvar o ID do áudio para a próxima etapa
audio_id = upload_data['id']

✅ Upload realizado com sucesso:
{
  "id": "5b950226-b404-41c1-bd3e-2d15a44811f8",
  "mimeType": null,
  "name": "audio-20260206-134341.wav",
  "size": 3272284,
  "url": "https://uploader.nativespeak.app/files/user_7c2798b8-0751-4fb6-aeb1-43b32842f600/meuprojeto/audio-20260206-134341.wav"
}


In [ ]:
# === Configurações do perfil ===
profile_name = 'Perfil PT-BR'
profile_language = 'pt-BR'
reference_text = 'Este é o texto de referência para treinar a voz'

# Mutation para criar perfil
profile_mutation = """
mutation($input: CreateProfileInput!) {
  createProfile(input: $input) {
      id
      name
      language
      referenceText
      audioFile {
            id
            url
            mimeType
            size
        }
    }
}
"""

profile_variables = {
    "input": {
        "name": profile_name,
        "language": profile_language,
        "referenceText": reference_text,
        "audioFileID": audio_id
    }
}

try:
    profile_resp = requests.post(url, headers=headers, json={
        'query': profile_mutation,
        'variables': profile_variables
    })
    profile_resp.raise_for_status()
    profile_data = profile_resp.json()['data']['createProfile']
    print("\n✅ Perfil criado com sucesso:")
    print(json.dumps(profile_data, indent=2))
except requests.exceptions.RequestException as e:
    print(f"Erro ao criar perfil: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")
    exit(1)


✅ Perfil criado com sucesso:
{
  "audioFile": {
    "id": "5b950226-b404-41c1-bd3e-2d15a44811f8",
    "mimeType": "audio/wav",
    "size": 3272284,
    "url": "https://uploader.nativespeak.app/files/user_7c2798b8-0751-4fb6-aeb1-43b32842f600/meuprojeto/audio-20260206-134341.wav"
  },
  "id": "5c749548-76be-4008-a9eb-54dba6351611",
  "language": "pt-BR",
  "name": "Perfil PT-BR",
  "referenceText": "Este \u00e9 o texto de refer\u00eancia para treinar a voz"
}


In [ ]:
# Exibir link do áudio e metadados
print("\n🔗 URL do áudio para playback ou download:")
print(profile_data['audioFile']['url'])

print("\n📄 Detalhes do áudio:")
print(f"ID: {profile_data['audioFile']['id']}")
print(f"MIME Type: {profile_data['audioFile']['mimeType']}")
print(f"Tamanho: {profile_data['audioFile']['size']} bytes")


🔗 URL do áudio para playback ou download:
https://uploader.nativespeak.app/files/user_7c2798b8-0751-4fb6-aeb1-43b32842f600/meuprojeto/audio-20260206-134341.wav

📄 Detalhes do áudio:
ID: 5b950226-b404-41c1-bd3e-2d15a44811f8
MIME Type: audio/wav
Tamanho: 3272284 bytes


In [ ]:
import requests
import json

# === Configurações ===
url = 'https://uploader.nativespeak.app/api/graphql'
# O token é obtido da célula anterior

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}

# Query para listar perfis com áudio completo
query = """
query {
  profiles {
    id
    name
    language
    referenceText
    audioFile {
      id
      url
      mimeType
      size
    }
  }
}
"""

payload = {'query': query}

try:
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    data = response.json()['data']['profiles']
    print(f"✅ {len(data)} perfis encontrados:\n")
    for p in data:
        print(f"ID do Perfil: {p['id']}")
        print(f"Nome: {p['name']}")
        print(f"Linguagem: {p['language']}")
        print(f"Texto de Referência: {p['referenceText']}")
        if p.get('audioFile'):
            print(f"  → Audio ID: {p['audioFile']['id']}")
            print(f"  → URL: {p['audioFile']['url']}")
            print(f"  → MIME: {p['audioFile']['mimeType']}")
            print(f"  → Tamanho: {p['audioFile']['size']} bytes")
        print("-" * 40)
except requests.exceptions.RequestException as e:
    print(f"Erro ao listar perfis: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")

✅ 2 perfis encontrados:

ID do Perfil: 621aee25-68f2-42c7-9639-9ca5eb158b24
Nome: Perfil PT-BR
Linguagem: pt-BR
Texto de Referência: Este é o texto de referência para treinar a voz
  → Audio ID: 5b950226-b404-41c1-bd3e-2d15a44811f8
  → URL: https://uploader.nativespeak.app/files/user_7c2798b8-0751-4fb6-aeb1-43b32842f600/meuprojeto/audio-20260206-134341.wav
  → MIME: audio/wav
  → Tamanho: 3272284 bytes
----------------------------------------
ID do Perfil: 5c749548-76be-4008-a9eb-54dba6351611
Nome: Perfil PT-BR
Linguagem: pt-BR
Texto de Referência: Este é o texto de referência para treinar a voz
  → Audio ID: 5b950226-b404-41c1-bd3e-2d15a44811f8
  → URL: https://uploader.nativespeak.app/files/user_7c2798b8-0751-4fb6-aeb1-43b32842f600/meuprojeto/audio-20260206-134341.wav
  → MIME: audio/wav
  → Tamanho: 3272284 bytes
----------------------------------------
